# Codechella

### Twitter Bot - Test

In [ ]:
import tweepy

##### Credentials

In [ ]:
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

###### Set up OAuth and integrate with API

In [ ]:
# auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_token_secret)
# # api = tweepy.API(auth)

# # create API object to pull data from twitter - and pass in authentication code
# api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [ ]:
# Write a tweet to push to our Twitter account
# Post a tweet from Python
# tweet = "Look, I'm tweeting from #Python in my #Codechella Hack @TwitterU"
# api.update_status(status=tweet)

It Works!!! 
---

# Historical Tweets Fetch

In [ ]:
# Install required packages
# !pip install GetOldTweets3 jsonpickle config pandas

In [ ]:
# Import all the required libraries
# import GetOldTweets3 as got
import pandas as pd
import tweepy
import shutil
# import jsonpickle
import json
import datetime
import time
import re
import csv

---

In [ ]:
"""
INPUTS:
    consumer_key, consumer_secret, access_token, access_token_secret: codes 
    telling twitter that we are authorized to access this data
    hashtag_phrase: the combination of hashtags to search for
OUTPUTS:
    none, simply save the tweet info to a spreadsheet
"""
def search_for_hashtags(consumer_key, consumer_secret, access_token, access_token_secret, hashtag_phrase):
    
    #create authentication for accessing Twitter
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

    #initialize Tweepy API
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    print('API created...')
    
    #get the name of the spreadsheet we will write to
    fname = '_'.join(re.findall(r"#(\w+)", hashtag_phrase))

    #open the spreadsheet we will write to
    with open('%s.csv' % (fname), 'w') as file:

        w = csv.writer(file)
        print('CSV File opened...')

        #write header row to spreadsheet
        # tweet Object - https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/overview/tweet-object (Data Dictionary)
        # user object - https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/user-object (Data Dictionary)
        # coordinates {gives Point coordinates <lat, long>} and places {gives Polygon coordinates <bounding box>} are different -> refer above tweet object link
        w.writerow(['tweet_id','timestamp','tweet_text','all_hashtags','coordinates','place','retweet_count', 'user_id','user_screenname','user_profile_description', 'user_profile_location']) 

        #for each tweet matching our hashtags, write relevant info to the spreadsheet
        for tweet in tweepy.Cursor(api.search, q=hashtag_phrase+' -filter:retweets', \
                                   lang="en", tweet_mode='extended', since='2010-11-21').items(5000):
            # print(tweet._json)
            # break
            coordinates = tweet.coordinates
            place = tweet._json['place']
            if coordinates:
                coordinates = coordinates['coordinates']
            if place:
                place = place['bounding_box']['coordinates']
            else:
                place = tweet.user.location

            w.writerow([
                tweet.id_str,
                tweet.created_at,
                tweet.full_text.replace('\n', ' '),
                [hashtag['text'] for hashtag in tweet._json['entities']['hashtags']],
                coordinates,
                place,
                tweet.retweet_count,
                tweet.user.id_str,
                tweet.user.screen_name,
                tweet.user.description.replace('\n',' '),
                tweet.user.location
            ])

In [ ]:
hashtag_phrase = input('Hashtag Phrase ') or '#save #help'
# hashtag_phrase = '#savelife #fund' #help #donatelife'

search_for_hashtags(consumer_key, consumer_secret, access_token, access_token_secret, hashtag_phrase)

Hashtag Phrase #serious #health
API created...
CSV File opened...


In [ ]:
# pd.read_csv("help.csv", header=0)